In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


Using BeautifulSoup to scrape the Data from the Wiki site

In [2]:
# librarys for web scraping
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = urlopen(url)

In [4]:
soup = BeautifulSoup(html,'lxml')
type(soup)

bs4.BeautifulSoup

In [5]:
# get the title
title = soup.title
print(title)

<title>List of postal codes of Canada: M - Wikipedia</title>


Only scraping the Table information because that's all we are interested in. Taking the rows and columns to form a CSV file to then convert to a DataFrame for easier manipulation.

In [6]:
Table = soup.find('table',{'class':'wikitable sortable'})
Table

Table1=""
for tr in Table.find_all('tr'):
    row1=""
    for tds in tr.find_all('td'):
        row1=row1+","+tds.text
    Table1=Table1+row1[1:]
print(Table1)

M1A,Not assigned,Not assigned
M2A,Not assigned,Not assigned
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Harbourfront
M5A,Downtown Toronto,Regent Park
M6A,North York,Lawrence Heights
M6A,North York,Lawrence Manor
M7A,Queen's Park,Not assigned
M8A,Not assigned,Not assigned
M9A,Etobicoke,Islington Avenue
M1B,Scarborough,Rouge
M1B,Scarborough,Malvern
M2B,Not assigned,Not assigned
M3B,North York,Don Mills North
M4B,East York,Woodbine Gardens
M4B,East York,Parkview Hill
M5B,Downtown Toronto,Ryerson
M5B,Downtown Toronto,Garden District
M6B,North York,Glencairn
M7B,Not assigned,Not assigned
M8B,Not assigned,Not assigned
M9B,Etobicoke,Cloverdale
M9B,Etobicoke,Islington
M9B,Etobicoke,Martin Grove
M9B,Etobicoke,Princess Gardens
M9B,Etobicoke,West Deane Park
M1C,Scarborough,Highland Creek
M1C,Scarborough,Rouge Hill
M1C,Scarborough,Port Union
M2C,Not assigned,Not assigned
M3C,North York,Flemingdon Park
M3C,North York,Don Mills South
M4C,East York,Woodbine Heights
M

In [8]:
file=open("toronto.csv","wb")
file.write(bytes(Table1,encoding="ascii",errors="ignore"))

df = pd.read_csv('toronto.csv', header = None)
df.columns=["Postalcode","Borough","Neighborhood"]

After reading in the CSV and labeling the columns, we need to get rid of the Not assigned values in the Borough Columns.

In [9]:
# getting indexes for when Borough = Not assigned
index_n = df[df['Borough'] == 'Not assigned'].index
# dropping these row indexes
df.drop(index_n,inplace=True)
df.head(10)

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


Giving the Not assigned values in the Neighborhood columns the value of the Borough for the same Postal code.

In [10]:
# Making not assigned Neighbourhoods Borough name
df.loc[df['Neighborhood'] == 'Not assigned','Neighborhood'] = df['Borough']
df.head(15)

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


Finaly grouping all of the Neighborhoods with the same Postal codes and Borough.

In [11]:
# grouping all Neighbourhoods with the same postal code
df_final = df.groupby(['Postalcode','Borough'], sort = False).agg(','.join)
df_final = df_final.reset_index()
df_final.head(12)

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


In [12]:
df_final.shape

(103, 3)

# Part 2: Adding Geographical Coordinates

Downloading and reading in the Csv file to add coordinates.

In [13]:
!wget -q -O 'Geospatial_data.csv' https://cocl.us/Geospatial_data
df_geo = pd.read_csv('Geospatial_data.csv')
df_geo.rename({'Postal Code':'Postalcode'},axis=1,inplace=True)
df_geo.head()

,Postalcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merging the df_final and df_geo to get a 5 column table with Postal codes, Borough, Neighborhoods, Latitude and Longitude.

In [14]:
df_geo_merged = pd.merge(df_final,df_geo[['Postalcode',
                                         'Latitude','Longitude']],
                         on='Postalcode')
df_geo_merged

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937


# Part 3: Exploring and Clustering Neighborhoods

In [15]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Getting the Coordinates for Toronto, ON. to later use for building a Toronto Map using the Folium package.

In [16]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude_toronto = location.latitude
longitude_toronto = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_toronto, longitude_toronto))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [17]:
Toronto_map = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=10)

# add markers to map
for lat, lng, borough, Neighborhood in zip(df_geo_merged['Latitude'], df_geo_merged['Longitude'], df_geo_merged['Borough'], df_geo_merged['Neighborhood']):
    label = '{}, {}'.format(Neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Toronto_map)  
    
Toronto_map

In [18]:
CLIENT_ID = '1YKWQQANL10FL5ZN2ZTDUKSZRLNY40J4GWKSAKPNYYTIPJM2' # your Foursquare ID
CLIENT_SECRET = 'JEEZCSSYQR2SQXQASWHJP5ZKXBSPDV45VELD5WQSFGBCSVI3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1YKWQQANL10FL5ZN2ZTDUKSZRLNY40J4GWKSAKPNYYTIPJM2
CLIENT_SECRET:JEEZCSSYQR2SQXQASWHJP5ZKXBSPDV45VELD5WQSFGBCSVI3


Exploring a random Neighborhood in the dataset. I chose Adelaide, King, and Richmond which happens to be in location 30 of the dataset and has lots of nearby venues. Then I will get the Latitude and the Longitude for that Neighborhood.

In [19]:
df_geo_merged.loc[30,'Neighborhood']

'Adelaide,King,Richmond'

In [20]:
neighborhood_latitude = df_geo_merged.loc[30, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_geo_merged.loc[30, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_geo_merged.loc[30, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Adelaide,King,Richmond are 43.65057120000001, -79.3845675.


This next cell give us the top 100 venues within a 500 meter radius and prints a url.

In [21]:
LIMIT = 100 #limiting the number of venues returned by Foursquare API
radius = 500 # define radius
url2 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url2 # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=1YKWQQANL10FL5ZN2ZTDUKSZRLNY40J4GWKSAKPNYYTIPJM2&client_secret=JEEZCSSYQR2SQXQASWHJP5ZKXBSPDV45VELD5WQSFGBCSVI3&v=20180605&ll=43.65057120000001,-79.3845675&radius=500&limit=100'

In [22]:
results = requests.get(url2).json()
results

{'meta': {'code': 200, 'requestId': '5cf5c41d4434b9214166c460'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Financial District',
  'headerFullLocation': 'Financial District, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 133,
  'suggestedBounds': {'ne': {'lat': 43.65507120450001,
    'lng': -79.37835988143604},
   'sw': {'lat': 43.64607119550001, 'lng': -79.39077511856397}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ad4c062f964a520e5f720e3',
       'name': 'Four Seasons Centre for the Performing Arts',
       'location': {'address': '145 Queen St. W',
        'crossStreet': 'at University Ave.',
        'lat': 43.650592,
        'lng': -79.38

In [23]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(20)

,name,categories,lat,lng
0,Four Seasons Centre for the Performing Arts,Concert Hall,43.650592,-79.385806
1,Nathan Phillips Square,Plaza,43.652270,-79.383516
2,The Keg Steakhouse & Bar,Steakhouse,43.649937,-79.384196
3,Shangri-La Toronto,Hotel,43.649129,-79.386557
4,Rosalinda,Vegetarian / Vegan Restaurant,43.650252,-79.385156
5,Canadian Opera Company,Opera House,43.650660,-79.386242
6,Bosk at Shangri-La,Asian Restaurant,43.649023,-79.385826
7,Soho House Toronto,Speakeasy,43.648734,-79.386541
8,Estiatorio Volos,Greek Restaurant,43.650329,-79.384533
9,John & Sons Oyster House,Seafood Restaurant,43.650656,-79.381613


In [24]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


Exploring all of the Neighborhoods in Toronto to see which Neighborhood has the most of a certain type of Venue.

In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
Toronto_venues = getNearbyVenues(names=df_geo_merged['Neighborhood'],
                                   latitudes=df_geo_merged['Latitude'],
                                   longitudes=df_geo_merged['Longitude']
                                  )

Parkwoods
Victoria Village
Harbourfront,Regent Park
Lawrence Heights,Lawrence Manor
Queen's Park
Islington Avenue
Rouge,Malvern
Don Mills North
Woodbine Gardens,Parkview Hill
Ryerson,Garden District
Glencairn
Cloverdale,Islington,Martin Grove,Princess Gardens,West Deane Park
Highland Creek,Rouge Hill,Port Union
Flemingdon Park,Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens,Eringate,Markland Wood,Old Burnhamthorpe
Guildwood,Morningside,West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor,Downsview North,Wilson Heights
Thorncliffe Park
Adelaide,King,Richmond
Dovercourt Village,Dufferin
Scarborough Village
Fairview,Henry Farm,Oriole
Northwood Park,York University
East Toronto
Harbourfront East,Toronto Islands,Union Station
Little Portugal,Trinity
East Birchmount Park,Ionview,Kennedy Park
Bayview Village
CFB Toronto,Downsview East
The Danforth West,Riverdale
Design E

In [27]:
print(Toronto_venues.shape)

(2256, 7)


Checking how many venues were returned for each neighborhood.

In [28]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",3,3,3,3,3,3
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",9,9,9,9,9,9
"Alderwood,Long Branch",10,10,10,10,10,10
"Bathurst Manor,Downsview North,Wilson Heights",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",22,22,22,22,22,22
Berczy Park,55,55,55,55,55,55


In [29]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 279 uniques categories.


In [30]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Wo

Grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [31]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Wo

Printing each Neighborhood with there top 5 most common venues.

In [32]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2      Thai Restaurant  0.04
3  American Restaurant  0.04
4           Steakhouse  0.04


----Agincourt----
                       venue  freq
0               Skating Rink  0.25
1                     Lounge  0.25
2             Breakfast Spot  0.25
3             Clothing Store  0.25
4  Middle Eastern Restaurant  0.00


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                       venue  freq
0                 Playground  0.33
1                       Park  0.33
2                Coffee Shop  0.33
3                Yoga Studio  0.00
4  Middle Eastern Restaurant  0.00


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
                  venue  freq
0         Grocery Store  0.22
1           Pizza Place  0.11
2           Coffee Shop  0.11
3        Sandwich Place  0.11
4  Fast Food Restaurant  0.1

           venue  freq
0  Shopping Mall   0.2
1           Park   0.2
2           Bank   0.2
3  Grocery Store   0.2
4          Hotel   0.2


----Downsview,North Park,Upwood Park----
                        venue  freq
0  Construction & Landscaping  0.25
1                        Park  0.25
2                      Bakery  0.25
3            Basketball Court  0.25
4                 Yoga Studio  0.00


----East Birchmount Park,Ionview,Kennedy Park----
               venue  freq
0     Discount Store  0.29
1         Hobby Shop  0.14
2  Convenience Store  0.14
3        Bus Station  0.14
4        Coffee Shop  0.14


----East Toronto----
                       venue  freq
0                       Park  0.50
1                Coffee Shop  0.25
2          Convenience Store  0.25
3                Yoga Studio  0.00
4  Middle Eastern Restaurant  0.00


----Emery,Humberlea----
                             venue  freq
0                   Baseball Field   1.0
1                      Yoga Studio   0.0
2      

                             venue  freq
0                              Spa   0.5
1                       Playground   0.5
2               Mexican Restaurant   0.0
3              Monument / Landmark   0.0
4  Molecular Gastronomy Restaurant   0.0


----St. James Town----
            venue  freq
0     Coffee Shop  0.07
1            Café  0.06
2           Hotel  0.05
3      Restaurant  0.05
4  Cosmetics Shop  0.04


----Stn A PO Boxes 25 The Esplanade----
                venue  freq
0         Coffee Shop  0.11
1          Restaurant  0.04
2                Café  0.04
3  Seafood Restaurant  0.03
4  Italian Restaurant  0.03


----Studio District----
                venue  freq
0                Café  0.11
1         Coffee Shop  0.08
2  Italian Restaurant  0.05
3              Bakery  0.05
4           Gastropub  0.05


----The Annex,North Midtown,Yorkville----
                           venue  freq
0                 Sandwich Place  0.12
1                           Café  0.12
2                   

Now to sort this data and put it into a pandas DataFrame.

In [33]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

This next cell sorts the data to give the top 10 venues for each Neighborhood

In [34]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Bar,American Restaurant,Thai Restaurant,Steakhouse,Hotel,Cosmetics Shop,Burger Joint,Bakery
1,Agincourt,Breakfast Spot,Lounge,Clothing Store,Skating Rink,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Coffee Shop,Playground,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Fried Chicken Joint,Coffee Shop,Pharmacy,Pizza Place,Beer Store,Fast Food Restaurant,Sandwich Place,Dog Run,Dim Sum Restaurant
4,"Alderwood,Long Branch",Pizza Place,Gym,Pharmacy,Sandwich Place,Pub,Pool,Dance Studio,Skating Rink,Coffee Shop,Drugstore


In [35]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 4, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 4, 0, 2,
       2, 2, 4, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 4, 4, 2, 2, 2, 4, 2, 2,
       2, 2, 2, 2, 2, 2, 1, 2, 4, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4], dtype=int32)

Creating a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [36]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster_Labels', kmeans.labels_)

Toronto_merged = df_geo_merged

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() # check the last columns!

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1.0,Fast Food Restaurant,Food & Drink Shop,Park,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
1,M4A,North York,Victoria Village,43.725882,-79.315572,2.0,Portuguese Restaurant,Coffee Shop,Pizza Place,Hockey Arena,Intersection,Women's Store,Dim Sum Restaurant,Diner,Discount Store,Dive Bar
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,2.0,Coffee Shop,Park,Pub,Bakery,Breakfast Spot,Restaurant,Mexican Restaurant,Café,Theater,Historic Site
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,2.0,Clothing Store,Women's Store,Coffee Shop,Boutique,Miscellaneous Shop,Furniture / Home Store,Gift Shop,Athletics & Sports,Event Space,Accessories Store
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,2.0,Coffee Shop,Gym,Park,Japanese Restaurant,Sushi Restaurant,Yoga Studio,Burger Joint,Italian Restaurant,Fast Food Restaurant,Smoothie Shop


Creating a Map of Toronto showing the Clusters

In [37]:
Toronto_merged = Toronto_merged.dropna()
Toronto_merged['Cluster_Labels'] = Toronto_merged.Cluster_Labels.astype(int)

# create map
map_clusters = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Cluster 1:

In [38]:
Toronto_merged.loc[Toronto_merged['Cluster_Labels'] == 0, 
                   Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,North York,0,Baseball Field,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Women's Store,Dim Sum Restaurant


Cluster 2:

In [40]:
Toronto_merged.loc[Toronto_merged['Cluster_Labels'] == 1,
                     Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,1,Fast Food Restaurant,Food & Drink Shop,Park,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
6,Scarborough,1,Fast Food Restaurant,Women's Store,Drugstore,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant


Cluster 3:

In [41]:
Toronto_merged.loc[Toronto_merged['Cluster_Labels'] == 2,
                     Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,2,Portuguese Restaurant,Coffee Shop,Pizza Place,Hockey Arena,Intersection,Women's Store,Dim Sum Restaurant,Diner,Discount Store,Dive Bar
2,Downtown Toronto,2,Coffee Shop,Park,Pub,Bakery,Breakfast Spot,Restaurant,Mexican Restaurant,Café,Theater,Historic Site
3,North York,2,Clothing Store,Women's Store,Coffee Shop,Boutique,Miscellaneous Shop,Furniture / Home Store,Gift Shop,Athletics & Sports,Event Space,Accessories Store
4,Queen's Park,2,Coffee Shop,Gym,Park,Japanese Restaurant,Sushi Restaurant,Yoga Studio,Burger Joint,Italian Restaurant,Fast Food Restaurant,Smoothie Shop
7,North York,2,Café,Gym / Fitness Center,Japanese Restaurant,Caribbean Restaurant,Basketball Court,Donut Shop,Diner,Discount Store,Dive Bar,Dog Run
8,East York,2,Fast Food Restaurant,Pizza Place,Gastropub,Pet Store,Pharmacy,Breakfast Spot,Bank,Intersection,Athletics & Sports,Gym / Fitness Center
9,Downtown Toronto,2,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Italian Restaurant,Theater,Sporting Goods Shop,Fast Food Restaurant,Restaurant
11,Etobicoke,2,Bank,Golf Course,Women's Store,Drugstore,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
12,Scarborough,2,History Museum,Bar,Women's Store,Drugstore,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
13,North York,2,Gym,Asian Restaurant,Coffee Shop,Beer Store,Clothing Store,Chinese Restaurant,Dim Sum Restaurant,Discount Store,Restaurant,Sandwich Place


Cluster 4:

In [42]:
Toronto_merged.loc[Toronto_merged['Cluster_Labels'] == 3,
                     Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,North York,3,Pizza Place,Empanada Restaurant,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant


Cluster 5:

In [43]:
Toronto_merged.loc[Toronto_merged['Cluster_Labels'] == 4,
                     Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,North York,4,Park,Pizza Place,Japanese Restaurant,Pub,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run
16,York,4,Park,Field,Hockey Arena,Trail,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run
21,York,4,Park,Fast Food Restaurant,Pharmacy,Market,Women's Store,Gourmet Shop,Grocery Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
35,East York,4,Park,Convenience Store,Coffee Shop,Drugstore,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
40,North York,4,Park,Other Repair Shop,Snack Place,Airport,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run
49,North York,4,Park,Bakery,Construction & Landscaping,Basketball Court,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
61,Central Toronto,4,Park,Swim School,Bus Line,Farmers Market,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
66,North York,4,Park,Bank,Convenience Store,Women's Store,Drugstore,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
68,Central Toronto,4,Jewelry Store,Trail,Park,Sushi Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
77,Etobicoke,4,Park,Pizza Place,Mobile Phone Shop,Bus Line,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doner Restaurant
